In [1]:
import os
import subprocess
import numpy as np
from astropy.io import fits

In [2]:
# File to store the consolidated results for all light curves
consolidated_file = "all_lc_periods_chisq_py_lc_files_orbit.txt"

# Create or clear the consolidated file
with open(consolidated_file, "w") as f:
    f.write("LC_File_Path\tPeriod\tChi-square\tDate\n")

# Find all obsID folders in the "output" directory
#obsid_folders = [f.path for f in os.scandir("./reduced_output") if f.is_dir() and f.name.startswith("6050")]
obsid_folders = sorted(
    [f.path for f in os.scandir("./reduced_output") if f.is_dir() and f.name.startswith("6050")],
    key=lambda x: int(os.path.basename(x))
)

# Loop through each obsID folder
for obsid_folder in obsid_folders:
    obsid = os.path.basename(obsid_folder)
    print(f"Processing obsID: {obsid}")

    # Define the light curve file path
    lc_files = []
    for root, dirs, files in os.walk(obsid_folder):
        for file in files:
            if file.endswith("barycorr_orbit_gti_0.05.lc"):
                lc_files.append(os.path.join(root, file))

    # Check if any light curve files were found
    if not lc_files:
        print(f"No light curve files found for obsID: {obsid}")
        continue

    # Loop through each light curve file
    for lc_file in lc_files:
        print(f"Processing light curve: {lc_file}")
        
        # Define paths for output files
        lc_folder = os.path.dirname(lc_file)
        output_folder = os.path.join(lc_folder, "efsearch_output")
        os.makedirs(output_folder, exist_ok=True)
        output_fits = os.path.join(output_folder, "efsearch_lc_orbit.fes")

        # Construct the efsearch command
        efsearch_cmd = f'efsearch {lc_file} window=- sepoch=20112.43540056619 dper=9.80 nphase=64 nbint=INDEF dres=1e-4 nper=1024 plot=no outfile={output_fits}'

        try:
            # Run the efsearch command
            subprocess.run(efsearch_cmd, shell=True, check=True)

            # Open the .fes FITS file and extract the best-fit period
            with fits.open(output_fits) as efsearch_fits:
                # Extract data from the 'RESULTS' table
                results_data = efsearch_fits['RESULTS'].data
                max_chisq_index = np.argmax(results_data['CHISQRD1'])  # Get index of maximum chi-square
                best_fit_period = results_data['PERIOD'][max_chisq_index]
                chisq_max = results_data['CHISQRD1'][max_chisq_index]

                # Extract the observation start time (TSTARTI) from the FITS header
                tstarti = efsearch_fits['RESULTS'].header.get('TSTARTI', 'N/A')  # 'N/A' if the keyword is missing

            # Append the results to the consolidated file
            with open(consolidated_file, "a") as f_out:
                f_out.write(f"{obsid}\t{best_fit_period}\t{chisq_max}\t{tstarti}\n")
            print(f"Processed {lc_file}, Best-fit period: {best_fit_period}, Chi-square: {chisq_max}, TSTARTI: {tstarti}")

        except subprocess.CalledProcessError as e:
            print(f"Error processing {lc_file}: {e}")

print(f"All periods, chi-squares, and TSTARTI values saved to {consolidated_file}")


Processing obsID: 6050390227
Processing light curve: ./reduced_output/6050390227/ni6050390227_cl_nightmpu7_sr_night_barycorr_orbit_gti_0.05.lc
 
efsearch 1.1 (xronos6.0)
 
 Series 1 file    1:./reduced_output/6050390227/ni6050390227_cl_nightmpu7_sr_nig
 
 Selected FITS extensions: 1 - RATE TABLE;
 
 Source ............ Swift_J0243.6+61    Start Time (d) .... 20112 10:25:07.184
 FITS Extension ....  1 - `RATE      `   Stop Time (d) ..... 20112 19:46:43.184
 No. of Rows .......        36480        Bin Time (s) ......   0.5000E-01
 Right Ascension ... 4.09188100E+01      Internal time sys.. Converted to TJD
 Declination ....... 6.14322700E+01      Experiment ........ NICER    XTI
 
 Corrections applied: Vignetting - No ; Deadtime - No ; Bkgd - No ; Clock - Yes
 
 Selected Columns:  1- Time;  2- Y-axis;  3- Y-error;  4- Fractional exposure;
 
 File contains binned data.
 
 
 Expected Start ... 20112.43411092593  (days)      10:25: 7:184  (h:m:s:ms)
 Expected Stop .... 20112.82411092593  (d